In [ ]:
!pip install -q tensorflow-addons  # version >= 0.15.0 is required
!pip install -q tensorflow
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 17.7 MB/s eta 0:00:00


In [ ]:
import copy

import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import datasets
from sklearn.metrics import classification_report, confusion_matrix

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [ ]:
TAG_SIZE = 2
VOCAB_SIZE = 20000

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def create_data_generatorr(dataset):
  def data_generator():
    for i in range(len(dataset)):
      left = dataset.loc[i]['x'].split(' ')
      right = None
      if dataset.loc[i]['y'] == 0:
        right = [0 for i in range(len(left))]
      else:
        right = [1 for i in range(len(left))]
      yield left, right
  
  return data_generator

df = pd.read_csv('/content/latest3.csv')
adf = df.copy()
ind = df[df['simple_sentence'].isna()].index
df.loc[ind, 'simple_sentence'] = df.loc[ind, 'claim']
df = df.sample(frac=1).reset_index(drop=True)
# grouped = adf.groupby('truth_value')

# Sample 900 rows from each group
# sampled = grouped.apply(lambda x: x.sample(n=566))

# # Reset the index of the sampled data
# sampled = sampled.reset_index(drop=True)
cdf = df.copy()
# df = sampled.copy()

def getds(col):
  global df
  le = LabelEncoder()
  df['truth_value'] = le.fit_transform(df['truth_value'])

  xtrain, xtest, ytrain, ytest = train_test_split(df[col], df['truth_value'], test_size=0.2, random_state=42, shuffle=False)

  train = pd.concat({'x': xtrain, 'y': ytrain}, axis=1)
  train.reset_index(drop=True, inplace=True)
  test = pd.concat({'x': xtest, 'y': ytest}, axis=1)
  test.reset_index(drop=True, inplace=True)

  splittrain = train['x'].apply(lambda x: x.split(' '))
  splittest = test['x'].apply(lambda x: x.split(' '))
  xtrain_tokens = tf.ragged.constant(train['x'].apply(lambda x: x.split(' ')))
  xtrain_tokens = tf.map_fn(tf.strings.lower, xtrain_tokens)
  xtest_tokens = tf.ragged.constant(test['x'].apply(lambda x: x.split(' ')))
  xtest_tokens = tf.map_fn(tf.strings.lower, xtest_tokens)

  trainlookup_layer = tf.keras.layers.StringLookup(max_tokens=VOCAB_SIZE, mask_token="[MASK]", oov_token="[UNK]")
  trainlookup_layer.adapt(xtrain_tokens)
  testlookup_layer = tf.keras.layers.StringLookup(max_tokens=VOCAB_SIZE, mask_token="[MASK]", oov_token="[UNK]")
  testlookup_layer.adapt(xtest_tokens)


  data_signature= (
          tf.TensorSpec(shape=(None,), dtype=tf.string),
          tf.TensorSpec(shape=(None, ), dtype=tf.int32)
  )

  train_data = tf.data.Dataset.from_generator(
      create_data_generatorr(train),
      output_signature=data_signature
  )
  test_data = tf.data.Dataset.from_generator(
      create_data_generatorr(test),
      output_signature=data_signature
  )
  return train_data, test_data, trainlookup_layer, testlookup_layer

tr1, te1, trll1, tell1 = getds('claim')
tr2, te2, trll2, tell2 = getds('simple_sentence')

In [ ]:
def dataset_preprocess_train1(tokens, tag_ids):
    preprocessed_tokens = preprecess_tokens_train1(tokens)
    return preprocessed_tokens, tag_ids

def preprecess_tokens_train1(tokens):
    tokens = tf.strings.lower(tokens)
    return trll1(tokens)

def dataset_preprocess_test1(tokens, tag_ids):
    preprocessed_tokens = preprecess_tokens_test1(tokens)
    return preprocessed_tokens, tag_ids

def preprecess_tokens_test1(tokens):
    tokens = tf.strings.lower(tokens)
    return tell1(tokens)

def dataset_preprocess_train2(tokens, tag_ids):
    preprocessed_tokens = preprecess_tokens_train2(tokens)
    return preprocessed_tokens, tag_ids

def preprecess_tokens_train2(tokens):
    tokens = tf.strings.lower(tokens)
    return trll2(tokens)

def dataset_preprocess_test2(tokens, tag_ids):
    preprocessed_tokens = preprecess_tokens_test2(tokens)
    return preprocessed_tokens, tag_ids

def preprecess_tokens_test2(tokens):
    tokens = tf.strings.lower(tokens)
    return tell2(tokens)

BATCH_SIZE = 10000

train_dataset1 = (
    tr1.map(dataset_preprocess_train1)
    .padded_batch(batch_size=BATCH_SIZE).cache()
)
test_dataset1 = (
    te1.map(dataset_preprocess_test1)
    .padded_batch(batch_size=BATCH_SIZE).cache()
)

train_dataset2 = (
    tr2.map(dataset_preprocess_train2)
    .padded_batch(batch_size=BATCH_SIZE).cache()
)
test_dataset2 = (
    te2.map(dataset_preprocess_test2)
    .padded_batch(batch_size=BATCH_SIZE).cache()
)


In [ ]:
# Build the model
def build_embedding_bilstm_crf_model(
    vocab_size: int, embed_dims: int, lstm_unit: int, tag_size: int
) -> tf.keras.Model:
    x = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="x")
    y = tf.keras.layers.Embedding(vocab_size, embed_dims, mask_zero=True)(x)
    y = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(lstm_unit, return_sequences=True)
    )(y)
    decode_sequence, potentials, sequence_length, kernel = tfa.layers.CRF(tag_size)(y)
    return tf.keras.Model(
        inputs=x, outputs=[decode_sequence, potentials, sequence_length, kernel]
    )


model1 = None
optimizer1 = None
model2 = None
optimizer2 = None

In [ ]:
@tf.function
def crf_loss_func(potentials, sequence_length, kernel, y):
    crf_likelihood, _ = tfa.text.crf_log_likelihood(
        potentials, y, sequence_length, kernel
    )
    # likelihood to loss
    flat_crf_loss = -1 * crf_likelihood
    crf_loss = tf.reduce_mean(flat_crf_loss)

    return crf_loss

train_loss1 = tf.keras.metrics.Mean(name="train_loss1")
train_loss2 = tf.keras.metrics.Mean(name="train_loss2")

@tf.function(experimental_relax_shapes=True)
def train_step(x, y):
    global model1
    global optimizer1
    with tf.GradientTape() as tape:
        decoded_sequence, potentials, sequence_length, kernel = model1(x)
        crf_loss = crf_loss_func(potentials, sequence_length, kernel, y)
        loss = crf_loss + tf.reduce_sum(model1.losses)
    grads = tape.gradient(loss, model1.trainable_variables)
    optimizer1.apply_gradients(zip(grads, model1.trainable_variables))

    train_loss1(loss)

@tf.function(experimental_relax_shapes=True)
def train_step2(x, y):
    global model2
    global optimizer2
    with tf.GradientTape() as tape:
        decoded_sequence, potentials, sequence_length, kernel = model2(x)
        crf_loss = crf_loss_func(potentials, sequence_length, kernel, y)
        loss = crf_loss + tf.reduce_sum(model2.losses)
    grads = tape.gradient(loss, model2.trainable_variables)
    optimizer2.apply_gradients(zip(grads, model2.trainable_variables))

    train_loss2(loss)


In [ ]:
model1 = build_embedding_bilstm_crf_model(VOCAB_SIZE, 32, 64, 2)
optimizer1 = tf.keras.optimizers.Adam(0.02)
model2 = build_embedding_bilstm_crf_model(VOCAB_SIZE, 32, 64, 2)
optimizer2 = tf.keras.optimizers.Adam(0.02)
tf.config.run_functions_eagerly(True)


for epoch in range(20):
  # Reset the metrics at the start of the next epoch
  train_loss1.reset_states()
  train_loss2.reset_states()

  for x, y in train_dataset1:
      train_step(x, y)
  for x, y in train_dataset2:
      train_step2(x, y)
  print(f"Epoch {epoch + 1}, " f"Loss1: {train_loss1.result()}")
  print(f"Epoch {epoch + 1}, " f"Loss2: {train_loss2.result()}")


Epoch 1, Loss1: 9.907398223876953
Epoch 1, Loss2: 27.276826858520508
Epoch 2, Loss1: 8.82288646697998
Epoch 2, Loss2: 24.2470703125
Epoch 3, Loss1: 9.924776077270508
Epoch 3, Loss2: 23.46659278869629
Epoch 4, Loss1: 7.072416305541992
Epoch 4, Loss2: 17.46314811706543
Epoch 5, Loss1: 6.746220111846924
Epoch 5, Loss2: 17.012495040893555
Epoch 6, Loss1: 5.6133341789245605
Epoch 6, Loss2: 13.219807624816895
Epoch 7, Loss1: 3.336562395095825
Epoch 7, Loss2: 8.478212356567383
Epoch 8, Loss1: 1.415023684501648
Epoch 8, Loss2: 5.919432163238525
Epoch 9, Loss1: 1.254758596420288
Epoch 9, Loss2: 4.747345924377441
Epoch 10, Loss1: 1.2020018100738525
Epoch 10, Loss2: 3.011951208114624
Epoch 11, Loss1: 0.855390191078186
Epoch 11, Loss2: 2.551292896270752
Epoch 12, Loss1: 0.4609343707561493
Epoch 12, Loss2: 2.3217475414276123
Epoch 13, Loss1: 0.30073678493499756
Epoch 13, Loss2: 1.7955296039581299
Epoch 14, Loss1: 0.22004500031471252
Epoch 14, Loss2: 1.3860732316970825
Epoch 15, Loss1: 0.20232470333

In [ ]:
with open('trll1.txt', 'w') as file:
  file.write('\n'.join(trll1.get_vocabulary(include_special_tokens=True)))
with open('trll2.txt', 'w') as file:
  file.write('\n'.join(trll2.get_vocabulary(include_special_tokens=True)))
with open('tell1.txt', 'w') as file:
  file.write('\n'.join(tell1.get_vocabulary(include_special_tokens=True)))
with open('tell2.txt', 'w') as file:
  file.write('\n'.join(tell2.get_vocabulary(include_special_tokens=True)))
model1.save('./nm1.h5')
model2.save('./nm2.h5')

In [ ]:
# train set
o1 = []
o2 = []
o = []
oy = []

def processs(x, pred):
  res = []
  for i in range(len(x)):
    b = pred[i]
    tot = (sum(b) / len(b)) * 100
    label = 1
    if tot <= 50:
      label = 0
    res.append(tot)
  return res

for x, y in train_dataset1:
  outputs, *_ = model1.predict(x)
  a = processs(x.numpy(), outputs)
  o1 += a
  oy += [p[0] for p in y.numpy()]
  
for x, y in train_dataset2:
  outputs, *_ = model2.predict(x)
  a = processs(x.numpy(), outputs)
  o2 += a

for i in range(len(o1)):
  o.append([o1[i].item() / 100, o2[i].item() / 100])


/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


302/302 [==============================] - 322s 1s/step


In [ ]:

# test set
to = []
toy = []

for x, y in test_dataset1:
  toy += [p[0] for p in y.numpy()]
  to += [[p[0], p[0]] for p in y.numpy()]

In [ ]:
len(np.array(o)), len(np.array(to))

(9645, 2412)

In [ ]:
import numpy as np

# tf.compat.v1.disable_eager_execution()
model4 = tf.keras.models.Sequential([
  tf.keras.layers.Dense(1, activation='sigmoid')
])
model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model4.fit(x=np.array(o), y=np.array(oy), validation_data=(np.array(to), np.array(toy)), epochs=10)

Epoch 1/10
302/302 [==============================] - 8s 27ms/step - loss: 0.5968 - accuracy: 0.9971 - val_loss: 0.3775 - val_accuracy: 1.0000
Epoch 2/10
302/302 [==============================] - 8s 28ms/step - loss: 0.5688 - accuracy: 0.9801 - val_loss: 0.3295 - val_accuracy: 1.0000
Epoch 3/10
302/302 [==============================] - 8s 26ms/step - loss: 0.5448 - accuracy: 0.9600 - val_loss: 0.2966 - val_accuracy: 1.0000
Epoch 4/10
302/302 [==============================] - 9s 30ms/step - loss: 0.5230 - accuracy: 0.9387 - val_loss: 0.2745 - val_accuracy: 1.0000
Epoch 5/10
302/302 [==============================] - 8s 27ms/step - loss: 0.5030 - accuracy: 0.9372 - val_loss: 0.2573 - val_accuracy: 1.0000
Epoch 6/10
302/302 [==============================] - 8s 26ms/step - loss: 0.4842 - accuracy: 0.9381 - val_loss: 0.2429 - val_accuracy: 1.0000
Epoch 7/10
302/302 [==============================] - 9s 29ms/step - loss: 0.4667 - accuracy: 0.9379 - val_loss: 0.2311 - val_accuracy: 1.0000

In [ ]:
m1ox = []
m1oy = []
m2ox = []
m2oy = []
fy = []

for x, y in test_dataset1:
  outputs, *_ = model1.predict(x)
  a = processs(x.numpy(), outputs)
  m1ox += a
  m1oy += [p[0] for p in y.numpy()]
  fy.extend([p[0] for p in y.numpy()])
  
for x, y in test_dataset2:
  outputs, *_ = model2.predict(x)
  m2ox = processs(x.numpy(), outputs)
  m2oy += a


/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


76/76 [==============================] - 45s 596ms/step


In [ ]:
report_model1_trainpredy = [int(i > 55) for i in o1]
report_model2_trainpredy = [int(i > 55) for i in o2]
report_model1_testpredy = [int(i > 55) for i in m1ox]
report_model2_testpredy = [int(i > 55) for i in m2ox]
# report_y = []
# for x, y in test_dataset1:
#   outputs, *_ = model1.predict(x)
#   a = processs(x.numpy(), outputs)
#   print(len(a), y.shape)

  
# for x, y in test_dataset2:
#   outputs, *_ = model2.predict(x)
#   a = processs(x.numpy(), outputs)
#   print(len(a), y.shape)

print(classification_report(oy, report_model1_trainpredy))
print(confusion_matrix(oy, report_model1_trainpredy), '\n\n')
print(classification_report(oy, report_model2_trainpredy))
print(confusion_matrix(oy, report_model2_trainpredy))

print(classification_report(fy, report_model1_testpredy))
print(confusion_matrix(fy, report_model1_testpredy), '\n\n')
print(classification_report(fy, report_model2_testpredy))
print(confusion_matrix(fy, report_model2_testpredy))



              precision    recall  f1-score   support

           0       0.53      1.00      0.69      5069
           1       1.00      0.01      0.03      4576

    accuracy                           0.53      9645
   macro avg       0.76      0.51      0.36      9645
weighted avg       0.75      0.53      0.38      9645

[[5069    0]
 [4508   68]] 


              precision    recall  f1-score   support

           0       0.53      1.00      0.69      5069
           1       1.00      0.00      0.01      4576

    accuracy                           0.53      9645
   macro avg       0.76      0.50      0.35      9645
weighted avg       0.75      0.53      0.37      9645

[[5069    0]
 [4560   16]]
              precision    recall  f1-score   support

           0       0.55      0.99      0.71      1250
           1       0.95      0.14      0.25      1162

    accuracy                           0.58      2412
   macro avg       0.75      0.57      0.48      2412
weighted avg     

In [ ]:
mox = []

for i in range(len(m1ox)):
  mox.append([m1ox[i].item() / 100, m2ox[i].item() / 100])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

for i in range(10, 95, 5):
  i /= 100
  print('for', i)
  outs = model4.predict(mox) > i
  outs = outs.flatten()

  print(classification_report(fy, outs))
  print(confusion_matrix(fy, outs), '\n\n')

for 0.1
46/76 [=================>............] - ETA: 0s

/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


76/76 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1250
           1       0.48      1.00      0.65      1162

    accuracy                           0.48      2412
   macro avg       0.24      0.50      0.33      2412
weighted avg       0.23      0.48      0.31      2412

[[   0 1250]
 [   0 1162]] 


for 0.15
46/76 [=================>............] - ETA: 0s

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/s

76/76 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1250
           1       0.48      1.00      0.65      1162

    accuracy                           0.48      2412
   macro avg       0.24      0.50      0.33      2412
weighted avg       0.23      0.48      0.31      2412

[[   0 1250]
 [   0 1162]] 


for 0.2
47/76 [=================>............] - ETA: 0s

...

76/76 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.61      0.98      0.75      1250
           1       0.95      0.32      0.48      1162

    accuracy                           0.67      2412
   macro avg       0.78      0.65      0.62      2412
weighted avg       0.77      0.67      0.62      2412

[[1231   19]
 [ 787  375]] 


for 0.85
47/76 [=================>............] - ETA: 0s

/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


76/76 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.59      0.99      0.74      1250
           1       0.96      0.26      0.41      1162

    accuracy                           0.64      2412
   macro avg       0.77      0.62      0.57      2412
weighted avg       0.77      0.64      0.58      2412

[[1237   13]
 [ 862  300]] 


for 0.9
44/76 [================>.............] - ETA: 0s

/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


76/76 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.56      1.00      0.72      1250
           1       0.97      0.17      0.29      1162

    accuracy                           0.60      2412
   macro avg       0.77      0.58      0.51      2412
weighted avg       0.76      0.60      0.51      2412

[[1244    6]
 [ 963  199]] 




In [ ]:
texmax, trxmax = 0, 0
for x, y in test_dataset1:
  # print(x.shape)
  texmax = max(texmax, x.shape[1])
print('\n')
for x, y in train_dataset1:
  # print(x.shape)
  trxmax = max(trxmax, x.shape[1])
print(texmax, trxmax)



60 81


In [ ]:
mo1 = tf.keras.models.load_model('/content/nm1.h5')
mo2 = tf.keras.models.load_model('/content/nm2.h5')

mo1oxtr = []
mo1oytr = []
mo2oxtr = []
mo2oytr = []
ftrysvm = []
mo1oxte = []
mo1oyte = []
mo2oxte = []
mo2oyte = []
fteysvm = []

def processs(x, pred):
  res = []
  for i in range(len(x)):
    b = pred[i]
    tot = (sum(b) / len(b)) * 100
    label = 1
    if tot <= 50:
      label = 0
    res.append(tot)
  return res

for x, y in train_dataset1:
  outputs, *_ = mo1.predict(x)
  a = processs(x.numpy(), outputs)
  mo1oxtr += a
  mo1oytr += [p[0] for p in y.numpy()]
  ftrysvm.extend([p[0] for p in y.numpy()])
  
for x, y in train_dataset2:
  outputs, *_ = mo2.predict(x)
  mo2oxtr = processs(x.numpy(), outputs)
  mo2oytr += a

for x, y in test_dataset1:
  outputs, *_ = mo1.predict(x)
  a = processs(x.numpy(), outputs)
  mo1oxte += a
  mo1oyte += [p[0] for p in y.numpy()]
  fteysvm.extend([p[0] for p in y.numpy()])
  
for x, y in test_dataset2:
  outputs, *_ = mo2.predict(x)
  mo2oxte = processs(x.numpy(), outputs)
  mo2oyte += a


/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


76/76 [==============================] - 44s 578ms/step


In [ ]:
svmtrx = None
svmtry = None
svmtex = None
svmtey = None
for x, y in train_dataset1:
  svmtrx = x
  svmtry = np.array([p[0] for p in y.numpy()])
for x, y in test_dataset1:
  svmtex = x
  svmtey = np.array([p[0] for p in y.numpy()])
svmtex = tf.pad(svmtex, [[0, 0], [0, svmtrx.shape[1] - svmtex.shape[1]]])

In [ ]:
len(mo1oxtr), len(mo1oytr), len(mo2oxtr), len(mo2oytr), len(mo1oxte), len(mo1oyte), len(mo2oxte), len(mo2oyte), svmtrx.shape, svmtex.shape, svmtry.shape, svmtey.shape

(9645,
 9645,
 9645,
 9645,
 2412,
 2412,
 2412,
 2412,
 TensorShape([9645, 81]),
 TensorShape([2412, 81]),
 (9645,),
 (2412,))

In [ ]:
nsvmtrx = []
nsvmtry = []
nsvmtex = []
nsvmtey = []

for i in range(svmtrx.shape[0]):
  nsvmtrx.append(list(svmtrx[i].numpy()) + [mo1oxtr[i], mo2oxtr[i]])
  nsvmtry.append(svmtry[i])
for i in range(svmtex.shape[0]):
  nsvmtex.append(list(svmtex[i].numpy()) + [mo1oxte[i], mo2oxte[i]])
  nsvmtey.append(svmtey[i])


In [ ]:
len(nsvmtrx), len(nsvmtrx[0]), len(nsvmtry), len(nsvmtex), len(nsvmtex[0]), len(nsvmtey)

(9645, 83, 9645, 2412, 83, 2412)

In [ ]:
from sklearn import svm

clf = svm.SVC(probability=True)
clf.fit(nsvmtrx, nsvmtry)

preds = clf.predict(nsvmtrx)

print(classification_report(nsvmtry, preds))
print(confusion_matrix(nsvmtry, preds))

              precision    recall  f1-score   support

           0       0.74      0.92      0.82      5069
           1       0.88      0.64      0.74      4576

    accuracy                           0.79      9645
   macro avg       0.81      0.78      0.78      9645
weighted avg       0.81      0.79      0.78      9645

[[4666  403]
 [1642 2934]]


In [ ]:
preds = clf.predict(nsvmtex)

print(classification_report(nsvmtey, preds))
print(confusion_matrix(nsvmtey, preds))

              precision    recall  f1-score   support

           0       0.72      0.90      0.80      1250
           1       0.85      0.62      0.72      1162

    accuracy                           0.77      2412
   macro avg       0.79      0.76      0.76      2412
weighted avg       0.78      0.77      0.76      2412

[[1125  125]
 [ 437  725]]
